"""
                          
    FileName:   cnn_model_aB9_v2       
    Date:       05/03/18 10:35 M   
    Author:     aB9           
                                
"""

In [1]:
# Import libraries
import os, shutil

from keras import models, optimizers
from keras import layers
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np

import tensorflow as tf

C:\Users\abhi0\Anaconda3\envs\py35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Gives available GPUs
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

# Creates a session with log_device_placement set to True.
# Check command prompt (terminal) for the output
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [3]:
# Directories
base_dir = os.path.normpath(os.getcwd() + os.sep + os.pardir+ os.sep + os.pardir)

dataset = os.path.join(base_dir,'dataset')
dataset_train =  os.path.join(dataset,'Training_Data')
dataset_test =  os.path.join(dataset,'Test_Data')
train_waterbodies = os.path.join(dataset_train,'water')
train_no_waterbodies =os.path.join(dataset_train,'waterless')
test_waterbodies = os.path.join(dataset_test, 'water')
test_no_waterbodies = os.path.join(dataset_test,  'waterless')


In [4]:
# Parameters
image_width, image_height = 256,256

In [5]:
# Creating the model (Sequential)
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), input_shape=(image_width, image_height, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Flatten the network
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 254, 32)      896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 127, 127, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 125, 125, 64)      18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 60, 60, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 28, 28, 128)       147584    
__________

In [6]:
# Configure model for training
model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=1e-4), metrics=['acc'])


In [ ]:
# Adding Data Augmentation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=90, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(dataset_train, target_size=(image_width,image_height), batch_size=32, class_mode='binary')

validation_generator = validation_datagen.flow_from_directory(dataset_test, target_size=(image_width,image_height), batch_size=32, class_mode='binary')

Found 1429 images belonging to 2 classes.
Found 358 images belonging to 2 classes.


In [ ]:
#Fitting the model
model.fit_generator(train_generator, steps_per_epoch=1000, epochs=5,validation_data=validation_generator, validation_steps=500)

Epoch 1/5
1000/1000 [==============================] - 971s 971ms/step - loss: 0.3604 - acc: 0.8557 - val_loss: 1.0986 - val_acc: 0.7427
Epoch 2/5
1000/1000 [==============================] - 1043s 1s/step - loss: 0.2062 - acc: 0.9283 - val_loss: 1.2512 - val_acc: 0.7373
Epoch 3/5
  87/1000 [=>............................] - ETA: 11:25 - loss: 0.1850 - acc: 0.9333

In [ ]:
h5_dir = os.path.join(base_dir,'h5')
model.save(h5_dir+os.sep+'cnn_model_aB9_v2.h5')